In [1]:
# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

2024-03-04 20:27:59.621237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Drop non-beneficial columns
application_df = application_df.drop(columns=["EIN", "NAME"])


In [3]:
# Binning categorical variables
# Binning APPLICATION_TYPE
app_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(app_counts[app_counts < 500].index)
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


In [4]:
# Binning CLASSIFICATION
class_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = list(class_counts[class_counts < 1800].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


In [5]:
# Convert categorical variables to numeric using one-hot encoding
encoded_df = pd.get_dummies(application_df)

# Split the preprocessed data into features and target arrays
X = encoded_df.drop(columns=["IS_SUCCESSFUL"])
y = encoded_df["IS_SUCCESSFUL"]

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [6]:
# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
# Define the model - deep neural net
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# Initialize the neural network model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5711 - accuracy: 0.7239
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7313
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7327
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7361
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5452 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7416
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.7403
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7417
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7405
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7405
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5355 - accuracy: 0.7398
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7405
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5348 - accuracy: 0.7400
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7402
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 

In [10]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5609 - accuracy: 0.7238 - 396ms/epoch - 1ms/step
Loss: 0.5609388947486877, Accuracy: 0.7238484025001526


In [11]:
# Save the model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/Users/aultsshit/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
